In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
bike_speed = 10
fly_speed = 15
start_distance = 20

In [3]:
class Mover():
    def __init__(self, pos, velocity, at=0):
        self.init_pos = pos - at * velocity
        self.velocity = velocity
        
    def position(self, t):
        return self.init_pos + self.velocity * t
    
    def collision_time(self, other):
        if abs(self.velocity - other.velocity) == 0.0:
            return None
        return (other.init_pos - self.init_pos) / (self.velocity - other.velocity)
    
    def turn_around_at(self, t):
        self.init_pos = self.position(t) + t * self.velocity
        self.init_time = t
        self.velocity = -self.velocity
        

In [10]:
bike1 = Mover(pos=0, velocity=bike_speed)
bike2 = Mover(pos=start_distance, velocity=-bike_speed)
fly = Mover(pos=0, velocity=fly_speed)

In [11]:
t = 0
fly_position = [(t, fly.position(t))]
fly_distance = 0.0

while bike1.position(t) < bike2.position(t) and len(fly_position) < 10:
    # find the time when the fly next encounters a bike
    t1 = fly.collision_time(bike1)
    t2 = fly.collision_time(bike2)
    if t1 > t:
        turn_t = t1
    else:
        turn_t = t2
    fly_position.append((turn_t, fly.position(turn_t)))
    fly_distance +=  abs(fly.position(turn_t) -  fly.position(t))
    fly.turn_around_at(turn_t)
    
    t = turn_t
    print(t, bike1.position(t), fly.position(t), bike2.position(t), fly_distance)

0.8 8.0 12.0 12.0 12.0
0.96 9.6 9.600000000000001 10.4 14.399999999999999
0.9919999999999999 9.919999999999998 10.080000000000002 10.080000000000002 14.879999999999999
0.9984000000000001 9.984 9.984 10.016 14.976
0.9984000000000002 9.984000000000002 9.984000000000002 10.015999999999998 14.976000000000003
0.9920000000000009 9.920000000000009 10.079999999999991 10.079999999999991 15.071999999999992
0.9920000000000009 9.920000000000009 10.079999999999991 10.079999999999991 15.071999999999992
0.9984000000000002 9.984000000000002 9.984000000000002 10.015999999999998 15.167999999999981
0.99968 9.9968 10.0032 10.0032 15.18719999999998


In [8]:
t = [item[0] for item in fly_position]
pfly = [item[1] for item in fly_position]
pbike1 = [bike1.position(ti) for ti in t]
pbike2 = [bike2.position(ti) for ti in t]

fig = figure(title="The Bikes and the Fly", x_axis_label='time (hours)', y_axis_label='position (miles)')

# add a line renderer with legend and line thickness
fig.line(t, pfly, legend="fly", line_width=2, line_color="red")
fig.line(t, pbike1, legend="bike1", line_width=2)
fig.line(t, pbike2, legend="bike2", line_width=2)

# show the results
show(fig)